_These are notes for some jQAssistant / Neo4j / Cypher demos_

# jQAssistant Demos
* Follow instructions here: https://github.com/JavaOnAutobahn/spring-petclinic/blob/master/readme.md
* Start the Neo4j server
 * `mvn jqassistant:server`
* Open up your browser
 * http://localhost:7474/browser/
* Open the drawer
* Click on some lables like
 * `Commit`
 * `Class`
 * `:DECLARES`
* Further information:
 * jQAssistant documentation: http://buschmais.github.io/jqassistant/doc/1.3.0/#_java_plugin
 
## Example Queries

**List types with the most methods**
```cypher
MATCH 
  (t:Type)-[:DECLARES]->(m:Method)
RETURN t.fqn as Typ, COUNT(m) as Methods
ORDER BY Methods DESC
```


**Find static, mutable fields**
```cypher
MATCH (c:Class)-[:DECLARES]->(f:Field)<-[w:WRITES]-(m:Method)
WHERE 
    EXISTS(f.static) AND NOT EXISTS(f.final)
RETURN 
    c.name as InClass, 
    m.name as theMethod, 
    w.lineNumber as writesInLine, 
    f.name as toStaticField
```

**Find recursive calls**
```cypher
MATCH
  (c:Class)-[:DECLARES]->(m:Method)-[:INVOKES*1..5]->(m)
RETURN c, m
```

**Find recursive calls to the database**
```cypher
MATCH 
  (m:Method)-[:INVOKES*]->(m)-[:INVOKES]->(dbMethod:Method),
  (dbMethod)<-[:DECLARES]-(dbClass:Class)
WHERE dbClass.name ENDS WITH "Database"
RETURN m, dbMethod, dbClass
```

**Aggregate measures to business subdomains**
```cypher
MATCH 
  (t:Type)-[:BELONGS_TO]->(s:Subdomain),
  (t)-[:HAS_CHANGE]->(ch:Change)
RETURN 
  s.name as ASubdomain,
  COUNT(DISTINCT t) as Types,
  COUNT(DISTINCT ch) as Changes
ORDER BY Types DESC
```


**Aggregate many measures to business subdomains**
```cypher
MATCH 
  (t:Type)-[:BELONGS_TO]->(s:Subdomain),
  (t)-[:HAS_CHANGE]->(ch:Change),
  (t)-[:HAS_MEASURE]->(co:Coverage)
OPTIONAL MATCH
  (t)-[:HAS_BUG]->(b:BugInstance)
RETURN 
  s.name as ASubdomain,
  COUNT(DISTINCT t) as Types,
  COUNT(DISTINCT ch) as Changes,
  AVG(co.ratio) as Coverage,
  COUNT(DISTINCT b) as Bugs,
  SUM(DISTINCT t.lastMethodLineNumber) as Lines
ORDER BY Coverage ASC, Bugs DESC
```